# What is this?

In 12-edo, all possible scales where all thirds are either major or minor thirds give us the most common, maybe the only scales used in western classical (extended tonality) music. Namely: major, melodic minor, harmonic minor, harmonic major, octatonic, wholetone, augmented, and their modes (rotations)

Similarly, we can try to do the same for 31-edo, or put a constraint on other intervals, or put constraints on multiple intervals at the same time.

In [ ]:
from collections import Counter, deque
from copy import copy
from itertools import accumulate, product

In [ ]:
def contains(this, other):
    return all([(a and b) or not a for a, b in zip(other, this)])


class Necklace(deque):
    def __eq__(self, other):
        assert len(self) == len(other)
        other = copy(other)
        for idx in range(len(self)):
            if deque.__eq__(self, other):
                return True
            other.rotate()
        return False

    def __ge__(self, other):
        assert len(self) == len(other)
        other = copy(other)
        for idx in range(len(self)):
            if contains(self, other):
                return True
            other.rotate()
        return False

    def __hash__(self):
        cache = [self.copy() for _ in range(len(self))]
        for n, val in enumerate(cache):
            val.rotate(n)
            cache[n] = tuple(val)

        return hash(frozenset(Counter(tuple(cache)).items()))

In [ ]:
def rotate_until_starts_with_one(s):
    s = copy(s)
    while s[0] != 1:
        s.rotate()
    return s

In [ ]:
def all_necklaces(k, n):
    all_lists = {min(i[j:] + i[:j] for j in range(n or 1)) for i in product(*[range(k)] * n)}
    return [rotate_until_starts_with_one(Necklace(s)) for s in all_lists if any(n == 1 for n in s)]

In [ ]:
def binary_to_delta(s):
    s = rotate_until_starts_with_one(s)
    return [len(a) + 1 for a in "".join(str(k) for k in s).split("1")[1:]]

In [ ]:
def get_intervals(scale, window):
    padded = scale + scale[: window - 1]
    intervals = [sum(padded[i : i + window]) for i in range(len(scale))]

    return intervals

In [ ]:
def is_valid(necklace, rules=[(2, [3, 4])]):
    # default rule is `interval between every other note is either major or minor third`
    scale = binary_to_delta(necklace)

    valid = True
    for rule in rules:
        window = rule[0]
        allowed = rule[1]
        intervals = get_intervals(scale, window)

        valid = valid and all((a in allowed) for a in intervals)

    return valid

In [ ]:
scales = all_necklaces(2, 12)

In [ ]:
quartal = [s for s in scales if is_valid(s, [(2, [2, 3, 4]), (3, [4, 5, 6])])]
triadic = [s for s in scales if is_valid(s)]

In [ ]:
exclusive = [q for q in quartal if q not in triadic]

In [ ]:
for e in exclusive:
    intervals = binary_to_delta(e)
    fourths = get_intervals(binary_to_delta(e), 2)

    print(intervals, [(fourths.count(s), s) for s in set(fourths)])

In [ ]:
len(quartal), len(exclusive)